# Association Analysis

Association rules are normally written like this: __{Diapers} -> {Beer}__ which means that there is a strong relationship between customers that purchased diapers and also purchased beer in the same transaction.

In the above example, the __{Diaper}__ is the antecedent and the __{Beer}__ is the consequent. Both antecedents and consequents can have multiple items. In other words, __{Diaper, Gum} -> {Beer, Chips}__ is a valid rule.

*** https://pbpython.com/market-basket-analysis.html
*** https://en.wikipedia.org/wiki/Association_rule_learning

## Support 
is the relative frequency that the rules show up. In many instances, you may want to look for high support in order to make sure it is a useful relationship. However, there may be instances where a low support is useful if you are trying to find “hidden” relationships.

## Confidence 
is a measure of the reliability of the rule. A confidence of .5 in the above example would mean that in 50% of the cases where Diaper and Gum were purchased, the purchase also included Beer and Chips. For product recommendation, a 50% confidence may be perfectly acceptable but in a medical situation, this level may not be high enough.

## Lift 
is the ratio of the observed support to that expected if the two rules were independent (see wikipedia). The basic rule of thumb is that a lift value close to 1 means the rules were completely independent. Lift values > 1 are generally more “interesting” and could be indicative of a useful rule pattern.

In [1]:
import pandas as pd
import numpy as np

In [11]:
! pip install --user mlxtend 

__Apriori__ is a popular algorithm for extracting frequent itemsets with applications in association rule learning. The apriori algorithm has been designed to operate on databases containing transactions, such as purchases by customers of a store. An itemset is considered as "frequent" if it meets a user-specified support threshold. For instance, if the support threshold is set to 0.5 (50%), a frequent itemset is defined as a set of items that occur together in at least 50% of all transactions in the database.

In [13]:
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [130]:
df=pd.read_excel('Online Retail.xlsx')

In [131]:
df.shape

(541909, 8)

In [132]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [133]:
# some of the descriptions have spaces that need to be removed. Drop the rows that don’t have invoice numbers and remove the credit transactions
df['Description']=df['Description'].str.strip()
df['InvoiceNo'].isna().any()

False

In [136]:
# find row index that are not credit transactions
index=~df['InvoiceNo'].astype(str).str.contains('C', case=False)

In [137]:
df=df.loc[index, :]

## Association analysis for France

In [138]:
# check country=France
df_fra=df.loc[df['Country']=='France',:]

In [148]:
df_fra_copy=df.loc[df['Country']=='France',:]

In [139]:
df_fra['InvoiceNo'].nunique()

392

In [140]:
pd.options.display.max_rows=None;
df_fra.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
26,536370,22728,ALARM CLOCK BAKELIKE PINK,24,2010-12-01 08:45:00,3.75,12583.0,France
27,536370,22727,ALARM CLOCK BAKELIKE RED,24,2010-12-01 08:45:00,3.75,12583.0,France
28,536370,22726,ALARM CLOCK BAKELIKE GREEN,12,2010-12-01 08:45:00,3.75,12583.0,France
29,536370,21724,PANDA AND BUNNIES STICKER SHEET,12,2010-12-01 08:45:00,0.85,12583.0,France
30,536370,21883,STARS GIFT TAPE,24,2010-12-01 08:45:00,0.65,12583.0,France


In [141]:
# drop duplicates with the same StockCode in each InvoiceNo 
df_fra=df_fra[~df_fra.loc[:,['InvoiceNo','StockCode']].duplicated()]

In [142]:
# Drop the duplicated with the same Description in each InvoiceNo

df_fra=df_fra[~df_fra.loc[:,['InvoiceNo','Description']].duplicated()]

In [143]:
# 1 use pivot_table to transform it into the right format 
pd.options.display.max_columns=None;

basket=df_fra.pivot_table(index='InvoiceNo', columns='Description',values='Quantity', aggfunc=pd.Series.nunique).fillna(0).convert_dtypes()

# 2 if we do not drop duplicated items before pivot_table, then use set_func to obtain 0 and 1 in basket
basket_copy=df_fra_copy.pivot_table(index='InvoiceNo', columns='Description',values='Quantity', aggfunc=np.mean).fillna(0).convert_dtypes()
# define function
def set_func(x):
    if x>0: 
        return 1
    else:
        return 0
    
basket_copy=basket_copy.applymap(set_func)
#return the items and itemsets with at least 7% support
basket_sets1=apriori(basket_copy, min_support=0.07, use_colnames=True )
basket_sets1.head()

In [144]:
basket.shape

(392, 1563)

In [161]:
#return the items and itemsets with at least 7% support
basket_sets=apriori(basket.drop(columns='POSTAGE'), min_support=0.07, use_colnames=True )
basket_sets.head()

,support,itemsets
0,0.071429,(4 TRADITIONAL SPINNING TOPS)
1,0.096939,(ALARM CLOCK BAKELIKE GREEN)
2,0.102041,(ALARM CLOCK BAKELIKE PINK)
3,0.094388,(ALARM CLOCK BAKELIKE RED)
4,0.081633,(BAKING SET 9 PIECE RETROSPOT)


In [169]:
# calculate confidence and lift
rules = association_rules(basket_sets, metric='lift', min_threshold=6)
rules.sort_values(by=['confidence','lift'], ascending= False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
11,"(SET/6 RED SPOTTY PAPER CUPS, SET/20 RED RETRO...",(SET/6 RED SPOTTY PAPER PLATES),0.102041,0.127551,0.099490,0.975000,7.644000,0.086474,34.897959
12,"(SET/6 RED SPOTTY PAPER PLATES, SET/20 RED RET...",(SET/6 RED SPOTTY PAPER CUPS),0.102041,0.137755,0.099490,0.975000,7.077778,0.085433,34.489796
9,(SET/6 RED SPOTTY PAPER PLATES),(SET/6 RED SPOTTY PAPER CUPS),0.127551,0.137755,0.122449,0.960000,6.968889,0.104878,21.556122
8,(SET/6 RED SPOTTY PAPER CUPS),(SET/6 RED SPOTTY PAPER PLATES),0.137755,0.127551,0.122449,0.888889,6.968889,0.104878,7.852041
3,(ALARM CLOCK BAKELIKE RED),(ALARM CLOCK BAKELIKE GREEN),0.094388,0.096939,0.079082,0.837838,8.642959,0.069932,5.568878
2,(ALARM CLOCK BAKELIKE GREEN),(ALARM CLOCK BAKELIKE RED),0.096939,0.094388,0.079082,0.815789,8.642959,0.069932,4.916181
10,"(SET/6 RED SPOTTY PAPER CUPS, SET/6 RED SPOTTY...",(SET/20 RED RETROSPOT PAPER NAPKINS),0.122449,0.132653,0.099490,0.812500,6.125000,0.083247,4.625850
6,(SET/6 RED SPOTTY PAPER PLATES),(SET/20 RED RETROSPOT PAPER NAPKINS),0.127551,0.132653,0.102041,0.800000,6.030769,0.085121,4.336735
5,(ALARM CLOCK BAKELIKE RED),(ALARM CLOCK BAKELIKE PINK),0.094388,0.102041,0.073980,0.783784,7.681081,0.064348,4.153061
14,(SET/6 RED SPOTTY PAPER PLATES),"(SET/6 RED SPOTTY PAPER CUPS, SET/20 RED RETRO...",0.127551,0.102041,0.099490,0.780000,7.644000,0.086474,4.081633


## Association analysis for AUSTRALIA

In [175]:
# check country=France
df_aus=df.loc[df['Country']=='Australia',:]
basket_aus=df_aus.pivot_table(index='InvoiceNo', columns='Description',values='Quantity', aggfunc=np.sum).fillna(0).convert_dtypes()
# drop postage column
basket_aus.drop(columns='POSTAGE', inplace=True)


In [187]:
# DEFINE unit_func
def unit_func(x):
    if x>0:
        return 1
    else:
        return 0
    
basket_aus  = basket_aus.applymap(unit_func)

# get itemsets and support
basket_aus_set=apriori(basket_aus, min_support=0.1, use_colnames=True )

In [188]:
rules_aus=association_rules(basket_aus_set, metric='confidence',  min_threshold=0.8)
rules_aus.sort_values(by=['confidence','lift'], ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(ALARM CLOCK BAKELIKE GREEN),(ALARM CLOCK BAKELIKE RED),0.105263,0.105263,0.105263,1.0,9.5,0.094183,inf
1,(ALARM CLOCK BAKELIKE RED),(ALARM CLOCK BAKELIKE GREEN),0.105263,0.105263,0.105263,1.0,9.5,0.094183,inf
2,(SPACEBOY LUNCH BOX),(DOLLY GIRL LUNCH BOX),0.105263,0.105263,0.105263,1.0,9.5,0.094183,inf
3,(DOLLY GIRL LUNCH BOX),(SPACEBOY LUNCH BOX),0.105263,0.105263,0.105263,1.0,9.5,0.094183,inf


### Observations
From 'rules_aus', it is interesting that SPACEBOY LUNCH BOX and DOLLY GIRL LUNCH BOX are purchased together.

ALARM CLOCK BAKELIKE GREEN and ALARM CLOCK BAKELIKE RED are purchased together.

Note that since the transaction number is limited in Australia, this may not provide convinceble insights here.

In [189]:
basket_aus['DOLLY GIRL LUNCH BOX'].sum()

6

In [186]:
basket_aus['SPACEBOY LUNCH BOX'].sum()

6